# Predicción de vuelos demorados

1. Ver qué atributos (evaluados con escala de 1 al 5) tienen mayor **correlación** con ser promotor de la marca. ¿Qué características del servicio valora más un promotor?
La hipótesis era que la demora en los vuelos pesaba más que otras variables. La demora sólo es controlable por la aerolínea al preparar la cabina y al coordinar a la tripulación.
VIZ: # y % de promotores en vuelos demorados y no demorados.
VIZ: Matriz de correlación
No pudimos definir correctamente el problema, por lo tanto se detiene el análisis desde el planteamiento.

2. Regresión logística para definir si un pasajero será promotor con base en

3. Herramienta para que el usuario tome decisiones informadas al comprar el vuelo.
Mostrar posibilidad de retraso en %
Insight: Las personas comentan que la comunicación acerca de es inoportuna o inexistente, por lo que al verlo previo a la compra pueden considerar otro horario de vuelo, otra aerolínea, reservaciones en hoteles, estructurar agendas, comprar alimentos.
Publicado por la aerolínea o en marca blanca: Despegar, etc.

Datset actual / real VivaAerobus (1 aerolínea):
15,500 de 5 meses
Variables:
Origen, Destino, Fecha y hora de salida programada, vuelo con demora, # de pasajeros con maleta documentada

Dataset USA:
607,000 en enero 2020
17 aerolíneas diferentes
Origen, Destino, Fecha y hora de salida programada, Fecha y hora de llegada programada, vuelo con demora en salida, vuelo con demora en llegada, causa de demora, vuelo cancelado, distancia y tiempo de vuelo

**Nos basamos en vuelos con demora solo en la salida, debido a retro del Viva**


## Preámbulo: Explorar dataset

1. Shape
2. Nulls
3. Variables

## Limpieza de datos
1. Quitar nulls
2. Imputar valores
3. Variables

## Estadística descriptiva V
- Barras >> \# y % de vuelos con demora y sin demora
- Barras >> a la salida o a la llegada*

### Horarios A
- BoxPlot >> X: 0 - 1, 1 - 2, ..., 23 - 24 Y: minutos de demora
- Tabla >> \# de vuelos con demora (>0) por hora del día, ordenado descendente

### Aerolíneas V
- Pie >> \# y % de vuelos por aerolínea
- Pie >> \# y % de vuelos demorados por aerolínea
A
- Tabla >> \# de vuelos con demora (>0) por aerolínea + media + max y min
- Barra >> \# de vuelos por grupo de demora (>0), 

### Ruta V
- Boxplot: Promedio por ciudad de origen + desviación estándar agrupada visualmente por ciudad de origen
A
- Heatmap: Promedio de retraso por día de la semana y ciudades de origen

### Aeropuerto
- Heatmap >> Promedio de minutos de retraso por aerolínea y aeropuerto, solo los 50 aeropuerto con más vuelos

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/vuelos-usa-ene2020/707333374_T_ONTIME_REPORTING.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'MONTH', 'CARRIER_DELAY', 'CANCELLATION_CODE', 'NAS_DELAY', 'Unnamed: 44',
                       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'CANCELLED', 'AIR_TIME']
df.drop(variables_to_remove, axis = 1, inplace = True)

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
df[["DEP_DELAY", "DAY_OF_WEEK"]].describe()

## Estadística descriptiva
___

In [ ]:
airlines_names = pd.read_csv('../input/airport/IATA_AIRLINE.asp')
airlines_names

In [ ]:
abbr_companies = airlines_names.set_index('Code')['Description'].to_dict()

In [ ]:
#Barras >> # y % de vuelos con demora y sin demora
sns.set(style="darkgrid")
is_delay = lambda x: 1 if x > 0 else 0
df['is_DELAY'] = df['DEP_DELAY'].apply(is_delay)
fig = plt.figure(1)
ax= sns.countplot(x="FLIGHTS", hue='is_DELAY', data=df)
plt.setp(ax.get_xticklabels(), fontsize=12, weight = 'normal', rotation = 0);
ax.yaxis.label.set_visible(False)
plt.xlabel('Retraso de Vuelos', fontsize=16, weight = 'bold', labelpad=10)
L = plt.legend()
L.get_texts()[0].set_text('Sin Retraso')
L.get_texts()[1].set_text('Con Retraso')
plt.show()

In [ ]:
#_____________________________________________
# Function that define how delays are grouped
delay_type = lambda x:((0,1)[x > 5],2)[x > 45]
df['DELAY_LEVEL'] = df['DEP_DELAY'].apply(delay_type)
#____________________________________________________
fig = plt.figure(1, figsize=(10,7))
ax = sns.countplot(y="OP_CARRIER", hue='DELAY_LEVEL', data=df)
#____________________________________________________________________________________
# We replace the abbreviations by the full names of the companies and set the labels
labels = [abbr_companies[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.setp(ax.get_xticklabels(), fontsize=12, weight = 'normal', rotation = 0);
plt.setp(ax.get_yticklabels(), fontsize=12, weight = 'bold', rotation = 0);
ax.yaxis.label.set_visible(False)
plt.xlabel('Conteo de Vuelos', fontsize=16, weight = 'bold', labelpad=10)
#________________
# Set the legend
L = plt.legend()
L.get_texts()[0].set_text('En tiempo (t < 5 min)')
L.get_texts()[1].set_text('Demora menor (5 < t < 45 min)')
L.get_texts()[2].set_text('Demora prolongada (t > 45 min)')
plt.show()

In [ ]:
#Barras >> a la salida o a la llegada*
sns.set(style="darkgrid")
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['hatch.linewidth'] = 2.0  

fig = plt.figure(1, figsize=(11,6))
ax = sns.barplot(x="DEP_DELAY", y="OP_CARRIER", data=df, color="lightskyblue", ci=None)
ax = sns.barplot(x="ARR_DELAY", y="OP_CARRIER", data=df, color="r", hatch = '///',alpha = 0.0, ci=None)
labels = [abbr_companies[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
ax.yaxis.label.set_visible(False)
plt.xlabel('Retraso Medio [min] (Salida: Azul, Llegada: Lineas de sombreado)',
           fontsize=14, weight = 'bold', labelpad=10);

### Aerolíneas

In [ ]:
#Pie >> # y % de vuelos por aerolínea
sns.set(style="darkgrid")
fig = plt.figure(1)
ax = sns.countplot(y="OP_CARRIER", hue='FLIGHTS', data=df)
labels = [abbr_companies[item.get_text()] for item in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.setp(ax.get_xticklabels(), fontsize=12, weight = 'normal', rotation = 0);
plt.setp(ax.get_yticklabels(), fontsize=12, weight = 'bold', rotation = 0);
ax.yaxis.label.set_visible(False)
plt.xlabel('Vuelos por Aerolinea', fontsize=16, weight = 'bold', labelpad=10)
plt.show()

In [ ]:
#__________________________________________________________________
# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean(), 
            'delay': (group > 0).sum(), 'not_delay': (group == 0).sum(), 'adel':(group < 0).sum()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = df['DEP_DELAY'].groupby(df['OP_CARRIER']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('mean')
global_stats

In [ ]:
import matplotlib.patches as mpatches
#_______________________________________________
df2 = df.loc[:, ['OP_CARRIER', 'FLIGHTS']]
df2['OP_CARRIER'] = df2['OP_CARRIER'].replace(abbr_companies)
#_______________________________________________
fig = plt.figure(1,figsize=(15,12))
gs=GridSpec(2,2)             
ax1=fig.add_subplot(gs[0,:]) 
ax2=fig.add_subplot(gs[1,:]) 
#_______________________________________________
labels = [s for s in  global_stats.index]
sizes  = global_stats['count'].values
explode = [0.3 if sizes[i] < 28000 else 0.0 for i in range(17)]
patches, texts, autotexts = ax1.pie(sizes, explode = explode,
                                labels=labels, autopct='%1.0f%%',
                                shadow=False, startangle=0)
#for i in range(17): 
#    texts[i].set_fontsize(14)
ax1.axis('equal')
ax1.set_title('% de Vuelos por Aerolinea', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
comp_handler = []
for i in range(17):
    comp_handler.append(mpatches.Patch(label = global_stats.index[i] + ': ' + abbr_companies[global_stats.index[i]]))
ax1.legend(handles=comp_handler, bbox_to_anchor=(0.2, 0.9), 
           fontsize = 8, bbox_transform=plt.gcf().transFigure)
#_______________________________________________
global_stats = global_stats.sort_values('delay')
sizes  = global_stats['delay'].values
explode = [0.3 if sizes[i] < 7000 else 0.0 for i in range(17)]
patches, texts, autotexts = ax2.pie(sizes, explode = explode,
                                labels=labels, autopct='%1.0f%%',
                                shadow=False, startangle=0)
#for i in range(17): 
#    texts[i].set_fontsize(14)
ax2.axis('equal')
ax2.set_title('% de Vuelos con retraso', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
plt.tight_layout() 

### Rutas

In [ ]:
#Heatmap: Promedio de retraso por día de la semana y ciudades de origen
 
airport_mean_delays = pd.DataFrame(pd.Series(df['ORIGIN_CITY_NAME'].unique()))
airport_mean_delays.set_index(0, drop = True, inplace = True)

for carrier in abbr_companies.keys():
    df1 = df[df['OP_CARRIER'] == carrier]
    test = df1['DEP_DELAY'].groupby(df['ORIGIN_CITY_NAME']).apply(get_stats).unstack()
    airport_mean_delays[carrier] = test.loc[:, 'mean'] 
    

sns.set(context="paper")
fig = plt.figure(1, figsize=(8,8))

ax = fig.add_subplot(1,2,1)
subset = airport_mean_delays.iloc[:50,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.01, cmap="Accent", mask=mask, vmin = 0, vmax = 35)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation = 85) ;
ax.yaxis.label.set_visible(False)

ax = fig.add_subplot(1,2,2)    
subset = airport_mean_delays.iloc[50:100,:].rename(columns = abbr_companies)
subset = subset.rename(index = identify_airport)
fig.text(0.5, 1.02, "Delays: impact of the origin airport", ha='center', fontsize = 18)
mask = subset.isnull()
sns.heatmap(subset, linewidths=0.01, cmap="Accent", mask=mask, vmin = 0, vmax = 35)
plt.setp(ax.get_xticklabels(), fontsize=10, rotation = 85) ;
ax.yaxis.label.set_visible(False)

plt.tight_layout()